In [33]:
import pandas as pd ; import numpy as np ; import statsmodels.api as sm
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data_train=pd.read_csv('./dataset_housing_preprocesado_train.csv')
data_test=pd.read_csv('./dataset_housing_preprocesado_test.csv')


In [34]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781 entries, 0 to 780
Columns: 153 entries, LotFrontage to Foundation_PConc
dtypes: float64(2), int64(151)
memory usage: 933.7 KB


In [35]:
del(data_train['Exterior_CBlock'])  #borramos la columna de ceros que no nos dejaba hacer la matriz de covarianza

Ahora procedemos a eliminar aquellas variables colineales, las detectaremos al ver en la matriz de correlación valores superiores a | 0.8 |, ya que nos ha parecido un valor óptimo para ver las colineales. Tiene que ser valor absoluto ya que pueder ser colineales y tener correlación negativa.

In [36]:
matriz_cor=data_train.corr()
data_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781 entries, 0 to 780
Columns: 152 entries, LotFrontage to Foundation_PConc
dtypes: float64(2), int64(150)
memory usage: 927.6 KB


In [37]:
data_train_aux = data_train
cols = data_train.columns
rank = np.linalg.matrix_rank(data_train.cov())

for col in cols:
    cols2 = cols.drop(col)
    data_train_aux = data_train_aux[cols2]
    rank2 = np.linalg.matrix_rank(data_train_aux.cov())
    if rank == rank2:
        cols = cols2 #si el rango es igual la columna era linealmente dep
    else:
        data_train_aux = data_train #si no habremos bajado un rango porq era indep


data_train = data_train[cols]
print(data_train.columns)
print(np.linalg.matrix_rank(data_train.cov()))

matriz_cor=data_train.cov()



Index(['LotFrontage', 'LotArea', 'LotShape', 'LandContour', 'LotConfig',
       'LandSlope', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       ...
       'HouseStyle_SFoyer', 'HouseStyle_SLvl', 'RoofStyle_Hip',
       'RoofStyle_Other', 'MasVnrType_BrkFace', 'MasVnrType_None',
       'MasVnrType_Stone', 'Foundation_CBlock', 'Foundation_Other',
       'Foundation_PConc'],
      dtype='object', length=140)
140


In [38]:
#Dividimos los datos entre la variable objetivo/dependiente (y) y las variables independientes
x_train = data_train.drop('SalePrice', axis=1)
y_train = data_train['SalePrice']
x_test  = data_test.drop("SalePrice", axis=1)
y_test  = data_test["SalePrice"]

SEMANA 2


Paso 1: Construcción de un modelo lineal con librería statsmodels


• Construye un modelo de regresión para el dataset anterior usando la libreríastatsmodels. ¿Obtiene los mismos coeficientes que sklearn? ¿Los coeficientes de determinación son iguales? ¿Es el QQ-plot el esperado para un modelo de regresión lineal?



Al construir el modelo de regresión lineal con statsmodels miramos que el coeficiente de determinación de model es del 0.991, por lo que vemos innecesario hacer una selección de variables. Esto también tiene que ver con el hecho de que nuestros datos ya han pasado por un preporocesado anterior. Si esto no fuese así, buscaríamos las variables cuyo p-valor fuese mayor que el nivel de significación elegido y las quitaríamos del modelo para tener un mayor coeficiente R2.

Los coeficientes son similares a sklearn son similares con valores absolutos pequeños, pero cuánto más grandes son estos más error puede llegar a ver entre ellos. Por ejemplo, los coeficientes de statsmodels y skelarn en LotShape son 1136 y 674.55,repectivamente. 

Como al ejecutar el QQPlot se visualiza el trazado ascendente de los residuos ordenados que conforman una clara línea, podemos confirmar que es la gráfica que esperábamos para el modelo de regresión lineal. La única pega son resiudos más altos, que se alejan de la recta. Esto quiere decir que la regresión lineal no se ajusta tanto para estos valores.



In [39]:
x=sm.add_constant(x_train)
model = sm.OLS(y_train, x_train).fit()






    

• Saca por pantalla la información estadística del modelo: desviación estándar de los coeficientes, valores p, intervalos de confianza de los coeficientes. Con una significancia estadística del 95%, ¿qué variables parece que no aportan
información?



Para detertar las variables que seguramente no aportan información tenemos que irnos a los p-valores más altos(siempre y cuando estos sean mayores que |t|), por lo que algunas de estas serían HouseStyle_1.5Unf, 3SsnPorch y MSSubClass_180.

In [40]:

#df_summary[df_summary['P>|t|']>0.05]
summary=model.summary()
df_summary = pd.read_html(summary.tables[1].as_html(), header=0, index_col=0)[0]



max=df_summary['P>|t|'].max()
        
fila_con_max_pvalue = df_summary.loc[df_summary['P>|t|'] == max]



    
    


C:\Users\juanj\AppData\Local\Temp\ipykernel_20592\132737248.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_summary = pd.read_html(summary.tables[1].as_html(), header=0, index_col=0)[0]


Paso 2: Selección de variables con statsmodels

• Implementa el siguiente algoritmo de eliminación de variables usando
statsmodels:

    1- Construye modelo con variables cols
    2- Detecta qué variables parece que no aportan información de acuerdo al
    criterio de los valores P. Si no encuentras ninguna, fin del algoritmo. Si encuentras una o varias, toma la que tenga un valor P mayor y quítala de cols. Vuelve a 1

In [48]:

summary=model.summary()
df_summary = pd.read_html(summary.tables[1].as_html(), header=0, index_col=0)[0]
#así podemos cargar en un Dataframe la segunda tabla que se nos despliega al hacer display(df.summary())


lista_cols_out=[]

while True:

    p_values=model.pvalues
    
    max=p_values.max()
    
    if max<=0.05:  #caso en el que ya no tengamos ningun pvalor mayor que el nivel de significación
        
        break
    
    lista_cols_out.append( model.pvalues.idxmax())
    model = sm.OLS(y_train, sm.add_constant(x_train.drop(lista_cols_out, axis=1))).fit()
    
    
summary=model.summary()
#display(summary)
df_summary = pd.read_html(summary.tables[1].as_html(), header=0, index_col=0)[0]



    

C:\Users\juanj\AppData\Local\Temp\ipykernel_20592\3226094777.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_summary = pd.read_html(summary.tables[1].as_html(), header=0, index_col=0)[0]
C:\Users\juanj\AppData\Local\Temp\ipykernel_20592\3226094777.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_summary = pd.read_html(summary.tables[1].as_html(), header=0, index_col=0)[0]


TypeError: object of type 'RegressionResultsWrapper' has no len()

• ¿Con cuántas variables te quedas usando este algoritmo? ¿Qué R2 tiene en training y en test el nuevo modelo? ¿Es mejor que el modelo sin seleccionar variables?


Paso 3: Selección de variables con regularización

• Crea un modelo lineal usando regresión Lasso usando la clase LassoCV del módulo linear_model de sklearn. ¿Qué variables tienen peso 0 en el modelo construido? ¿es esto consistente con lo que has visto en el paso 1? ¿Qué R2 en training y test tiene tu modelo? ¿es mejor respecto a los anteriores?
 Importante: para usar regresión Lasso o Ridge, las variables de entrada al modelo y la variable target se deben estandarizar.

• Repite el punto anterior usando regresión Ridge